In [76]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import matplotlib.pyplot as plt
warnings.simplefilter('ignore')

In [354]:
ob_df = pd.read_csv('/Users/seanyu_mac/Downloads/아워박스_통합관리_조회_250327 - 시트1.csv')
fs_df = pd.read_excel('/Users/seanyu_mac/Downloads/주문내역_2024-03-27~2025-03-27 (1).xlsx')

In [356]:
# 두개 테이블의 주문 일시와 결제 완료일은 datetime 형식으로 변환 후 str 타입으로 변환
ob_df['주문일시'] = pd.to_datetime(ob_df['주문일시'])
ob_df = ob_df.sort_values(by='주문일시', ascending=False)
ob_df = ob_df.reset_index()
fs_df['결제 완료일'] = pd.to_datetime(fs_df['결제 완료일'])
fs_df = fs_df.sort_values(by='결제 완료일', ascending=False)
fs_df = fs_df.reset_index()

fs_df['결제 완료일'] = fs_df['결제 완료일'].dt.strftime('%Y-%m-%d %H:%M:%S')
fs_df['결제 완료일']
ob_df['주문일시'] = ob_df['주문일시'].astype(str)
ob_df['주문일시']

# 두 테이블의 우편번호를 str 타입으로 변환
ob_df['우편번호'] = ob_df['우편번호'].astype(str)
fs_df['우편번호'] = fs_df['우편번호'].fillna(0)
fs_df['우편번호'] = fs_df['우편번호'].astype(int)
fs_df['우편번호'] = fs_df['우편번호'].astype(str)
fs_df['우편번호']

# 두 테이블의 주문번호를 앞 5자리 추출 후 str 타입으로 변환
ob_df['주문번호'] = ob_df['주문번호'].apply(lambda x: x[:5])
ob_df['주문번호']
fs_df['주문 번호'] = fs_df['주문 번호'].apply(lambda x: x[:5])

# 아워박스 데이터의 결제 금액을 뜻하는 sales 컬럼 생성 후 str 타입으로 변환
ob_df['sales'] = ob_df['수량'] * ob_df['판매단가']
ob_df['sales'] = ob_df['sales'].astype(str)
fs_df['정산가'] = fs_df['정산가'].astype(str)

# 머지하기 위한 key 컬럼을 생성 : 주문일시 + 우편번호 + 주문번호 + 결제금액
# ob_df['key'] = ob_df['주문일시'] + ob_df['우편번호'] + ob_df['주문번호'] + ob_df['sales']
ob_df['key'] = ob_df['우편번호'] + ob_df['주문번호'] + ob_df['sales']
# fs_df['key'] = fs_df['결제 완료일'] + fs_df['우편번호'] + fs_df['주문 번호'] + fs_df['정산가']
fs_df['key'] = fs_df['우편번호'] + fs_df['주문 번호'] + fs_df['정산가']


# 머지하여 temp_df 생성하고 html 태그 제거 후 엑셀 다운로드
temp_df = pd.merge(ob_df, fs_df[['key', '옵션명','상품명']], on='key', how='left')
datas = []

for i in temp_df['품목명']:
    if '>S' in i:
        data = i.split('>S')[1].strip()
    elif '>D' in i:
        data = i.split('>D')[1].strip()
    datas.append(data)
temp_df['품목명'] = datas
temp_df['채널상품명'] = temp_df['상품명']
temp_df['옵션1'] = temp_df['옵션명']

temp_df['품목명'] = temp_df['품목명'].str.replace('</span>', '')
temp_df['품목명'] = temp_df['품목명'].apply(lambda x: x.strip())

temp_df['도서산간여부'] = temp_df['도서산간여부'].str.replace("<span class='square square-gray'>", '')
temp_df['도서산간여부'] = temp_df['도서산간여부'].str.replace("</span>", '')
temp_df['도서산간여부'] = temp_df['도서산간여부'].apply(lambda x: x.strip())

temp_df['주문서상태'] = temp_df['주문서상태'].str.replace("<span class='square square-primary'>", '')
temp_df['주문서상태'] = temp_df['주문서상태'].str.replace("<span class='square square-warning'>", '')
temp_df['주문서상태'] = temp_df['주문서상태'].str.replace("</span>", '')
temp_df['주문서상태'] = temp_df['주문서상태'].apply(lambda x: x.strip())
temp_df['주문서상태']

del temp_df['key']
del temp_df['상품명']
del temp_df['옵션명']
del temp_df['sales']
del temp_df['index']

today = datetime.now().strftime('%Y-%m-%d')
temp_df.to_excel(f'/Users/seanyu_mac/Downloads/아워박스_상품명_옵션명_업데이트_{today}.xlsx')

In [366]:
fs_df['주문 상태'].isin(['배송준비', '배송중', '배송완료', '구매확정'])
fs_df['정산가'] = fs_df['정산가'].astype(int)
fs_df['정산가'].sum()

125822940